<a href="https://colab.research.google.com/github/omkar-salunke/accounts/blob/main/SOE_TALLY_Daybook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# New Section

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [69]:
# prompt: list of files in the path in python

import os
import pandas as pd
path = '/content/drive/My Drive/2024_25/SOE/'  # Replace with your desired path
files = os.listdir(path)
print(files)


['SOE.XML', 'SOE1.xml']


In [29]:
df_bank_match = pd.read_excel("/content/drive/My Drive/2024_25/overview_account.xlsx",sheet_name='SOE 2024-25')
df_bank_match_overview = df_bank_match.groupby('comment').agg({'Debit':'sum','Credit':'sum'}).reset_index().rename(columns={'comment':'party_name'})

In [49]:
import xml.etree.ElementTree as ET
import pandas as pd

# Function to recursively extract text and sub-elements from a node
def extract_element_data(element):
    element_data = {}
    # If the element has no children, return its text
    if len(element) == 0:
        return element.text

    # Otherwise, go deeper into the element's children
    for child in element:
        # Recursively extract data for each child element
        child_data = extract_element_data(child)
        element_data[child.tag] = child_data
    return element_data

# Function to flatten nested dictionaries (e.g., 'amount': {'P': {'T': 100}})
def flatten_dict(d, parent_key='', sep='.'):
    items = []
    for k, v in d.items():
        new_key = f"{parent_key}{sep}{k}" if parent_key else k
        if isinstance(v, dict):
            items.extend(flatten_dict(v, new_key, sep=sep).items())
        else:
            items.append((new_key, v))
    return dict(items)

# Function to recursively search for all 'voucher' elements and extract their data
def find_all_vouchers(element, voucher_tag="VOUCHER"):
    vouchers = []

    # If the current element is a 'voucher', collect its data
    if element.tag == voucher_tag:
        voucher_dict = {}
        # Collect data for all child elements within the voucher
        for sub_child in element:
            extracted_data = extract_element_data(sub_child)
            voucher_dict[sub_child.tag] = extracted_data

        # Flatten the voucher dictionary to handle nested elements
        voucher_dict_flattened = flatten_dict(voucher_dict)
        vouchers.append(voucher_dict_flattened)

    # Recursively search in all children of the current element
    for child in element:
        vouchers.extend(find_all_vouchers(child, voucher_tag))

    return vouchers

# Function to convert the XML to a DataFrame by extracting all vouchers
def xml_to_dataframe_vouchers(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()

    # Find all 'voucher' elements in the XML
    voucher_list = find_all_vouchers(root)

    # Create a DataFrame from the list of voucher dictionaries
    df = pd.DataFrame(voucher_list)
    return df


In [89]:
df_raw_collect = pd.DataFrame()
req_cols = ['DATE','GUID','VOUCHERTYPENAME','PARTYLEDGERNAME','PARTYGSTIN','CLASSNAME','PARTYNAME','CONSIGNEEGSTIN','PERSISTEDVIEW','ALLINVENTORYENTRIES.LIST.STOCKITEMNAME','ALLINVENTORYENTRIES.LIST.RATE','ALLINVENTORYENTRIES.LIST.AMOUNT','LEDGERENTRIES.LIST.VATEXPAMOUNT','LEDGERENTRIES.LIST.AMOUNT','ALLINVENTORYENTRIES.LIST.ACTUALQTY','ALLINVENTORYENTRIES.LIST.BILLEDQTY','ALLINVENTORYENTRIES.LIST.BATCHALLOCATIONS.LIST.AMOUNT','ALLINVENTORYENTRIES.LIST.BATCHALLOCATIONS.LIST.ACTUALQTY','ALLINVENTORYENTRIES.LIST.BATCHALLOCATIONS.LIST.BILLEDQTY','ALLINVENTORYENTRIES.LIST.ACCOUNTINGALLOCATIONS.LIST.LEDGERNAME','ALLINVENTORYENTRIES.LIST.ACCOUNTINGALLOCATIONS.LIST.AMOUNT','LEDGERENTRIES.LIST.LEDGERNAME','LEDGERENTRIES.LIST.AMOUNT','LEDGERENTRIES.LIST.VATEXPAMOUNT','INVOICEORDERLIST.LIST.BASICORDERDATE','INVOICEORDERLIST.LIST.BASICPURCHASEORDERNO','REFERENCEDATE','REFERENCE','CONSIGNEEPINNUMBER','ALLINVENTORYENTRIES.LIST.ACCOUNTINGALLOCATIONS.LIST.GSTOVRDNNATURE','ALLINVENTORYENTRIES.LIST.ACCOUNTINGALLOCATIONS.LIST.RATEDETAILS.LIST.GSTRATEDUTYHEAD','ALLLEDGERENTRIES.LIST.OLDAUDITENTRYIDS.LIST.OLDAUDITENTRYIDS','ALLLEDGERENTRIES.LIST.LEDGERNAME','ALLLEDGERENTRIES.LIST.AMOUNT','ALLLEDGERENTRIES.LIST.BANKALLOCATIONS.LIST.DATE','ALLLEDGERENTRIES.LIST.BANKALLOCATIONS.LIST.INSTRUMENTDATE','ALLLEDGERENTRIES.LIST.BANKALLOCATIONS.LIST.NAME','ALLLEDGERENTRIES.LIST.BANKALLOCATIONS.LIST.TRANSACTIONTYPE','ALLLEDGERENTRIES.LIST.BANKALLOCATIONS.LIST.PAYMENTFAVOURING','ALLLEDGERENTRIES.LIST.BANKALLOCATIONS.LIST.CHEQUECROSSCOMMENT','ALLLEDGERENTRIES.LIST.BANKALLOCATIONS.LIST.UNIQUEREFERENCENUMBER','ALLLEDGERENTRIES.LIST.BANKALLOCATIONS.LIST.STATUS','ALLLEDGERENTRIES.LIST.BANKALLOCATIONS.LIST.PAYMENTMODE','ALLLEDGERENTRIES.LIST.BANKALLOCATIONS.LIST.BANKPARTYNAME','ALLLEDGERENTRIES.LIST.BANKALLOCATIONS.LIST.AMOUNT','NARRATION']

for xt in files:
  path_ct = '/content/drive/My Drive/2024_25/SOE/'
  xml_file = path_ct+xt
  df_raw_f = xml_to_dataframe_vouchers(xml_file)[req_cols].reset_index(drop=True)
  df_raw_f_ref = pd.DataFrame(df_raw_f)
  df_raw_collect = pd.concat([df_raw_collect, df_raw_f_ref]).reset_index(drop=True)

df_raw = df_raw_collect.drop_duplicates(subset='GUID', keep='first').reset_index(drop=True)

In [94]:
df = df_raw[req_cols]

In [96]:
df['party_name'] = df[['PARTYNAME', 'ALLLEDGERENTRIES.LIST.BANKALLOCATIONS.LIST.PAYMENTFAVOURING', 'ALLLEDGERENTRIES.LIST.LEDGERNAME']].bfill(axis=1).iloc[:, 0]
df['tax_out_1'] = df[['LEDGERENTRIES.LIST.AMOUNT']].bfill(axis=1).iloc[:, 0]
df['tax_out_2'] = df[['LEDGERENTRIES.LIST.VATEXPAMOUNT']].bfill(axis=1).iloc[:, 0]

# df[''] = df[''].astype(float)
df['ALLINVENTORYENTRIES.LIST.BATCHALLOCATIONS.LIST.AMOUNT'] = df['ALLINVENTORYENTRIES.LIST.BATCHALLOCATIONS.LIST.AMOUNT'].astype(float)
df['tax_out_1'] = df['tax_out_1'].astype(float)
df['tax_out_2'] = df['tax_out_2'].astype(float)

df['amount_coalesce'] = df[['ALLINVENTORYENTRIES.LIST.BATCHALLOCATIONS.LIST.AMOUNT', 'ALLLEDGERENTRIES.LIST.AMOUNT']].bfill(axis=1).iloc[:, 0]
df['amount_coalesce'] = df['amount_coalesce'].astype(float)
df['tax_coalesce'] = df[['tax_out_1', 'tax_out_2']].sum(axis=1)
df['total_coalesce'] = df[['tax_coalesce', 'amount_coalesce']].sum(axis=1)

df = df[['party_name','total_coalesce','amount_coalesce','tax_coalesce','tax_out_1','tax_out_2']+req_cols]
df

<ipython-input-96-8aec564b4acb>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['party_name'] = df[['PARTYNAME', 'ALLLEDGERENTRIES.LIST.BANKALLOCATIONS.LIST.PAYMENTFAVOURING', 'ALLLEDGERENTRIES.LIST.LEDGERNAME']].bfill(axis=1).iloc[:, 0]
<ipython-input-96-8aec564b4acb>:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['tax_out_1'] = df[['LEDGERENTRIES.LIST.AMOUNT']].bfill(axis=1).iloc[:, 0]
<ipython-input-96-8aec564b4acb>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using

,party_name,total_coalesce,amount_coalesce,tax_coalesce,tax_out_1,tax_out_2,DATE,GUID,VOUCHERTYPENAME,PARTYLEDGERNAME,...,ALLLEDGERENTRIES.LIST.BANKALLOCATIONS.LIST.NAME,ALLLEDGERENTRIES.LIST.BANKALLOCATIONS.LIST.TRANSACTIONTYPE,ALLLEDGERENTRIES.LIST.BANKALLOCATIONS.LIST.PAYMENTFAVOURING,ALLLEDGERENTRIES.LIST.BANKALLOCATIONS.LIST.CHEQUECROSSCOMMENT,ALLLEDGERENTRIES.LIST.BANKALLOCATIONS.LIST.UNIQUEREFERENCENUMBER,ALLLEDGERENTRIES.LIST.BANKALLOCATIONS.LIST.STATUS,ALLLEDGERENTRIES.LIST.BANKALLOCATIONS.LIST.PAYMENTMODE,ALLLEDGERENTRIES.LIST.BANKALLOCATIONS.LIST.BANKPARTYNAME,ALLLEDGERENTRIES.LIST.BANKALLOCATIONS.LIST.AMOUNT,NARRATION
0,AKAR INDUSTRY,44765.42,37936.8,6828.62,3414.31,3414.31,20240401,c00e677e-43c1-4bea-8147-e2e4905eacab-0000b597,SALES GST,AKAR INDUSTRY,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AKAR INDUSTRY,36464.84,30902.4,5562.44,2781.22,2781.22,20240401,c00e677e-43c1-4bea-8147-e2e4905eacab-0000b598,SALES GST,AKAR INDUSTRY,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AKAR INDUSTRY,23267.84,18178.0,5089.84,2544.92,2544.92,20240401,c00e677e-43c1-4bea-8147-e2e4905eacab-0000b599,SALES GST,AKAR INDUSTRY,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,V.S.AUTO TECH PRIVATE LIMITED,186487.20,158040.0,28447.20,14223.60,14223.60,20240401,c00e677e-43c1-4bea-8147-e2e4905eacab-0000b59a,SALES GST,V.S.AUTO TECH PRIVATE LIMITED,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,V.S.AUTO TECH PRIVATE LIMITED,54257.58,45981.0,8276.58,4138.29,4138.29,20240401,c00e677e-43c1-4bea-8147-e2e4905eacab-0000b59b,SALES GST,V.S.AUTO TECH PRIVATE LIMITED,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1305,AKAR INDUSTRY,24679.70,20915.0,3764.70,1882.35,1882.35,20241030,c00e677e-43c1-4bea-8147-e2e4905eacab-0000bc33,SALES GST,AKAR INDUSTRY,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1306,AKAR INDUSTRY,34302.60,29070.0,5232.60,2616.30,2616.30,20241030,c00e677e-43c1-4bea-8147-e2e4905eacab-0000bc34,SALES GST,AKAR INDUSTRY,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1307,AKAR INDUSTRY,31436.80,24560.0,6876.80,3438.40,3438.40,20241030,c00e677e-43c1-4bea-8147-e2e4905eacab-0000bc35,SALES GST,AKAR INDUSTRY,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1308,AKAR INDUSTRY,19107.16,16192.5,2914.66,1457.33,1457.33,20241030,c00e677e-43c1-4bea-8147-e2e4905eacab-0000bc36,SALES GST,AKAR INDUSTRY,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [126]:
df[df['party_name']=='A.B.ENGINEERING WORKS']['total_coalesce'].sum()#[['total_coalesce','amount_coalesce','tax_coalesce','tax_out_1','tax_out_2','LEDGERENTRIES.LIST.VATEXPAMOUNT','LEDGERENTRIES.LIST.AMOUNT']]


23446.92

In [98]:
# df.groupby("party_name").pivot("VOUCHERTYPENAME")['amount_coalesce'].sum()
df['total_coalesce'] = df['total_coalesce'].astype('float')
df_summary_all = pd.pivot_table(df, values='total_coalesce', index='party_name', columns='VOUCHERTYPENAME', aggfunc='sum').reset_index()
df_summary_all['balance'] = 0
df_summary_all.loc[df_summary_all[df_summary_all['party_name']=='TIRUMALA MULTI TECHNOLOGIES'].index,'balance'] = 467646
df_summary_all.loc[df_summary_all[df_summary_all['party_name']=='SAVITA AUTO INDUSTRIES (N)'].index,'balance'] = 195853
df_summary_all.loc[df_summary_all[df_summary_all['party_name']=='PATIL-N-PATIL ENGINEERING	'].index,'balance'] = 55660
df_summary_all.loc[df_summary_all[df_summary_all['party_name']=='NANDAMURI TECHNOS'].index,'balance'] = 35255
df_summary_all.loc[df_summary_all[df_summary_all['party_name']=='V.S.AUTO TECH PRIVATE LIMITED'].index,'balance'] = 4699956
df_summary_all.loc[df_summary_all[df_summary_all['party_name']=='A.B.ENGINEERING WORKS'].index,'balance'] = 11598

df_summary_all.loc[df_summary_all[df_summary_all['party_name']=='AKAR INDUSTRY'].index,'balance'] = 317899

df_summary_all.loc[df_summary_all[df_summary_all['party_name']=='SURYA AUTO INDUSTRIES'].index,'balance'] = 23165


df_summary_all = df_summary_all.fillna(0)
df_summary_all['total_due']=df_summary_all[['Contra', 'Credit Note', 'DELIVERY CHALLAN', 'Debit Note',
       'Journal', 'Payment', 'Purchase', 'Receipt', 'SALES GST', 'balance']].sum(axis=1)
df_summary_all

VOUCHERTYPENAME,party_name,Contra,Credit Note,DELIVERY CHALLAN,Debit Note,Journal,Payment,Purchase,Receipt,SALES GST,balance,total_due
0,A.B.ENGINEERING WORKS,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,23446.92,11598,35044.92
1,AKAR INDUSTRY,0.0,0.0,0.0,81446.64,0.0,0.0,0.00,-2831903.0,5923540.12,317899,3490982.76
2,ALPINE CORPORATION,0.0,0.0,0.0,0.00,0.0,0.0,-1094.92,0.0,0.00,0,-1094.92
3,ASHISH TRADING COMPANY,0.0,0.0,0.0,0.00,0.0,0.0,-2505.58,0.0,0.00,0,-2505.58
4,AURANGABAD STEEL CORPORATION,0.0,0.0,0.0,0.00,0.0,0.0,0.00,-2023953.7,2690636.00,0,666682.30
...,...,...,...,...,...,...,...,...,...,...,...,...
59,TIRUMALA MULTI TECHNOLOGIES,0.0,0.0,0.0,0.00,892.0,0.0,-2808.60,-533309.0,23538.84,467646,-44040.76
60,V.S.AUTO TECH PRIVATE LIMITED,0.0,0.0,0.0,0.00,0.0,0.0,0.00,-7312871.2,8870318.67,4699956,6257403.47
61,VAIBHAVSHREE TRADING,0.0,0.0,0.0,0.00,0.0,5000.0,0.00,0.0,0.00,0,5000.00
62,VEDANT METALLICS PVT LTD,0.0,0.0,0.0,0.00,0.0,5976479.0,-5427369.95,0.0,0.00,0,549109.05


In [118]:
# df_bank_match
df_summary_all_bank = pd.merge(df_summary_all,df_bank_match_overview,on=['party_name'],how='outer')
df_summary_all_bank = df_summary_all_bank.sort_values(by='SALES GST', key=lambda x: x.isna()).reset_index(drop=True).sort_values(by='SALES GST', ascending=False).reset_index(drop=True)


In [121]:
import numpy as np
columns_to_round = ['Contra', 'Credit Note', 'DELIVERY CHALLAN', 'Debit Note',
       'Journal', 'Payment', 'Purchase', 'Receipt', 'SALES GST', 'balance',
       'total_due', 'Debit', 'Credit']
df_summary_all_bank[columns_to_round] = df_summary_all_bank[columns_to_round].apply(np.ceil)


In [122]:
df_summary_all_bank

,party_name,Contra,Credit Note,DELIVERY CHALLAN,Debit Note,Journal,Payment,Purchase,Receipt,SALES GST,balance,total_due,Debit,Credit
22,V.S.AUTO TECH PRIVATE LIMITED,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-7312871.0,8870319.0,4699956.0,6257404.0,0.0,7312872.0
6,PATIL-N-PATIL ENGINEERING,0.0,-292079.0,0.0,457615.0,0.0,0.0,-7468333.0,0.0,6490900.0,0.0,-811899.0,NaN,NaN
53,AKAR INDUSTRY,0.0,0.0,0.0,81447.0,0.0,0.0,0.0,-2831903.0,5923541.0,317899.0,3490983.0,0.0,2831893.0
50,AURANGABAD STEEL CORPORATION,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-2023953.0,2690636.0,0.0,666683.0,0.0,2023954.0
19,SAVITA AUTO INDUSTRIES (N),0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1619006.0,2232818.0,195853.0,809665.0,0.0,1619006.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86,GAVALI VASUDEO TANAJI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,225511.0,0.0
87,PRASHANT S,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,268800.0,0.0
88,GST,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,192.0,0.0
89,MAYURI SALUNKES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100000.0,0.0


In [123]:
df.to_excel("/content/drive/My Drive/2024_25/SOE_daybook.xlsx",sheet_name='master_sheet')

In [124]:

with pd.ExcelWriter("/content/drive/My Drive/2024_25/SOE_daybook.xlsx",mode='a') as writer:
  df_summary_all_bank.to_excel(writer,sheet_name='summary')
with pd.ExcelWriter("/content/drive/My Drive/2024_25/SOE_daybook.xlsx",mode='a') as writer:
  df_raw.to_excel(writer,sheet_name='raw_xml')
# for xrow in df['VOUCHERTYPENAME'].unique().tolist():
#   with pd.ExcelWriter("/content/drive/My Drive/2024_25/SOE_daybook.xlsx",mode='a') as writer:
#     df[df['VOUCHERTYPENAME']==xrow].reset_index(drop=True).to_excel(writer,sheet_name=xrow)


In [ ]:
# import os
# from datetime import datetime
# path = '/content/drive/My Drive/2024_25'  # Replace with your desired path
# files = os.listdir(path)
# old_file_path  = '/content/drive/My Drive/2024_25/SOEDayBook.xlsx'
# today_date = datetime.today().strftime('%Y-%m-%d')
# if 'SOEDayBook.xlsx'  in files:
#     # Create the new file name by appending today's date
#   file_name, file_extension = os.path.splitext(old_file_path)
#   new_file_path = f"{file_name}_{today_date}{file_extension}"

#   # Rename the file
#   os.rename(old_file_path, new_file_path)
#   if not os.path.exists(old_file_path) and os.path.exists(new_file_path):
#       print(f"File renamed and saved as: {new_file_path}")
#       df.to_excel("/content/drive/My Drive/2024_25/SOE_daybook.xlsx",sheet_name='master_sheet')

#       for xrow in df['VOUCHERTYPENAME'].unique().tolist():
#         with pd.ExcelWriter("/content/drive/My Drive/2024_25/SOE_daybook.xlsx",mode='a') as writer:
#           df[df['VOUCHERTYPENAME']==xrow].reset_index(drop=True).to_excel(writer,sheet_name=xrow)
#   else:
#       print(f"Error renaming file: {old_file_path}")

In [ ]:
# import pandas as pd
# df['DATE'] = pd.to_datetime(df['DATE'])

# df_sales = df[df['VOUCHERTYPENAME']=='SALES GST'].reset_index(drop=True)
# # Calculate new dates
# df_sales['DATE_plus_45'] = df_sales['DATE'] + pd.DateOffset(days=45)
# df_sales['DATE_plus_60'] = df_sales['DATE'] + pd.DateOffset(days=60)
# df_sales['DATE_plus_90'] = df_sales['DATE'] + pd.DateOffset(days=90)
# df_sales['Week_Number'] = df_sales['DATE_plus_45'].dt.isocalendar().week
# df_sales['ALLINVENTORYENTRIES.LIST.AMOUNT'] = df_sales['ALLINVENTORYENTRIES.LIST.AMOUNT'].astype(float)
# df_sales = df_sales.rename(columns={'ALLINVENTORYENTRIES.LIST.AMOUNT': 'sales_amount'})
# df_sales_g = df_sales.groupby(['PARTYLEDGERNAME','Week_Number'])['sales_amount'].sum().reset_index()

# # Add a column for the payment due date (45 days after the sale date)
# df_sales['due_date'] = df_sales['DATE'] + pd.Timedelta(days=45)

# # Filter the DataFrame to include only pending payments (due date is in the future)
# pending_payments = df_sales[df_sales['due_date'] > pd.Timestamp.now()]

# # Group by customer name and week of the year, and sum the amounts
# pending_payments_per_week = pending_payments.groupby(['PARTYLEDGERNAME'])['sales_amount'].sum().reset_index()

# pending_payments_per_week.columns = ['PARTYLEDGERNAME', 'due_amount_'+str(pd.Timestamp.now().date())]
# outstanding_payments_g = df_sales.groupby(['PARTYLEDGERNAME'])['sales_amount'].sum().reset_index()

# outstanding_payments_g.columns = ['PARTYLEDGERNAME', 'outstanding_amount']
# df_receipt = df[df['VOUCHERTYPENAME']=='Receipt'].reset_index(drop=True)
# df_receipt['ALLLEDGERENTRIES.LIST.AMOUNT'] = df_receipt['ALLLEDGERENTRIES.LIST.AMOUNT'].astype(float)
# df_receipt = df_receipt.rename(columns={'ALLLEDGERENTRIES.LIST.AMOUNT': 'receipt_amount'})
# df_receipt_g = df_receipt.groupby(['PARTYLEDGERNAME'])['receipt_amount'].sum().reset_index()
# df_due_45_days = pd.merge(df_receipt_g,outstanding_payments_g,on=['PARTYLEDGERNAME'],how='right')
# df_due_45_days = pd.merge(df_due_45_days,pending_payments_per_week,on=['PARTYLEDGERNAME'],how='left')
# df_due_45_days['closing_balance_on_31_march'] = ''
# with pd.ExcelWriter("/content/drive/My Drive/2024_25/SOE_daybook.xlsx",mode='a') as writer:
#   df_due_45_days.to_excel(writer,sheet_name='Due_45_days')


/usr/local/lib/python3.10/dist-packages/openpyxl/packaging/relationship.py:118: UserWarning: xl/pivotCache/_rels/pivotCacheDefinition1.xml.rels contains invalid dependency definitions
  warn(msg)


ValueError: Sheet 'Due_45_days' already exists and if_sheet_exists is set to 'error'.